In [1]:
from pathlib import Path
import torch
torch.manual_seed(42)

In [2]:
### Point to files
rootdir = Path(f'/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/for_pytorch_wide_5_704_32')
# rootdir = Path(f'/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/masked_for_pytorch_wide_1_512_32')
flist = list(rootdir.glob('*.pt'))
flist.sort()

traindir = Path(rootdir, 'train')
traindir.mkdir(exist_ok=True)
testdir = Path(rootdir, 'test')
testdir.mkdir(exist_ok=True)
valdir = Path(rootdir, 'val')
valdir.mkdir(exist_ok=True)
flistdir = Path(rootdir, 'flist')
flistdir.mkdir(exist_ok=True)

In [3]:
### Split up files
## Counting
pct_train, pct_val = 0.9, 0.1
assert pct_train+pct_val==1, "Percents don't add up!"

n_tot = len(flist)
n_train = int(pct_train*n_tot)+1
n_val = int(pct_val*n_tot)
assert n_train+n_val==n_tot, f"Splits don't add up! {n_train+n_val}!={n_tot} Manually partition instead."

## Split up files
generator = torch.Generator().manual_seed(42)
train_list, val_list = torch.utils.data.random_split(flist, [n_train, n_val], generator=generator)
train_list, val_list = list(train_list), list(val_list)

train_list.sort()
val_list.sort()

## Move files and create flists
with open(Path(flistdir, 'train.flist'), 'w') as f:
    for file in train_list:
        file.rename(Path(traindir, file.name))
        f.write(f'{Path(traindir, file.name)}\n')

with open(Path(flistdir, 'val.flist'), 'w') as f:
    for file in val_list:
        file.rename(Path(valdir, file.name))
        f.write(f'{Path(valdir, file.name)}\n')

In [4]:
foo = torch.load(Path(traindir, 'sample_0000_00.pt'))

In [5]:
foo['x'].shape

torch.Size([5, 704, 32])

In [6]:
foo['y'].shape

torch.Size([5, 704, 32])